전처리

-------------------------------
<br />

모든 팀원이 쉽게 이해할 수 있는 코드를 작성하여 효율적인 프로젝트 진행을 목표로 합니다.
<br />
<br />
**변수 생성 기준**

- 예측 정확도 상승
- 편의

**객체(변수, 함수) 네이밍 규칙**

- 고유명사가 아닌 줄임말은 명확하도록 풀어 적는다. e.g.) STT -> sched_time
- 줄임말 기준: 
    - 줄임말을 썻을 때 이해하기 쉬운 경우
    - 기존 이름이 너무 장황
    - 일반적으로 많이 쓰는 줄임말을 사용. 
    - e.g.) count -> cnt, average -> avg, schedule -> sched 
- 동사를 앞에 적는다. 동사\_명사 e.g.) sum_flight, avg_flight, cnt_flight
- 자주 사용되는 단어일수록 앞에 위치 e.g.) sched_time, sched_datetime
- 더미변수(binary 변수)는 is_이름 e.g.) is_delay, is_holiday
- 생소하거나 어려운 단어라면 길더라도 그냥 쓰자.
- "."을 사용하지말고 "\_"로 통일하자. "."을 사용하게 되면 메소드와 혼돈을 일으킴.
- GPA와 같은 이름을 제외하면 소문자를 사용하자.
- 영어 이름을 사용하자.

**가독성**

- 주석을 달아주자.
- 코딩실력이 된다면 혹은 코딩실력 상승을 위해서
    - 코드 실행 단축을 위한 벡터화: 간단히 설명하자면 for루프, apply 대신 판다스, 사이킨런 내장함수를 사용하면 된다.
    - 반복된 코드로 스크립트가 길어지는 것을 방지하기 위해 함수화하자. 
    - 메모리를 위해 가끔식 객체 삭제가 필요할 수도 있다.
- 파이썬 스타일 가이드 PEP8을 지키자. 적어도 망치를 눌러주자.

<br />

모든 데이터셋에 대해서 다음과 같은 변수를 추가했습니다. 

- 날짜(date): yyyy-mm-dd
- 날짜시간(datetime): yyyy-mm-dd hh:mm:ss

EDA를 기준으로 작성하였으며 모델링용 데이터셋은 나중에 다시 작성하겠습니다.

# AFSNT.csv

In [59]:
import pandas as pd

df_afsnt = pd.read_csv("AFSNT.csv", encoding="CP949")
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

df_afsnt.head()

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,sched_time,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,...,10:05:00,10:32:00,N,NaN,N,NaN,2017-01-01,2017-01-01 10:05:00,2017-01-01 10:32:00,27
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,...,09:30:00,09:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 09:30:00,2017-01-01 09:31:00,1
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,...,12:45:00,13:03:00,N,NaN,N,NaN,2017-01-01,2017-01-01 12:45:00,2017-01-01 13:03:00,18
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,...,13:25:00,14:09:00,Y,C02,N,NaN,2017-01-01,2017-01-01 13:25:00,2017-01-01 14:09:00,44
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,...,16:10:00,16:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 16:10:00,2017-01-01 16:31:00,21


# AFSNT_DLY.csv

In [62]:
df_test = pd.read_csv("AFSNT_DLY.csv", encoding="CP949")
df_test["STT"] = pd.to_datetime(df_test["STT"], format = "%H:%M").dt.time

df_test = df_test.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", 
    "DLY":"is_delay", "DLY_RATE":"prob_delay"
})
df_test["date"] = pd.to_datetime(df_test[["year", "month", "day"]])
df_test["sched_datetime"] = pd.to_datetime(df_test['date'].astype(str) + " " + df_test["sched_time"].astype(str))

df_test.head()

,year,month,day,wday,origin,dest,airline,flight,is_arrive,sched_time,is_delay,prob_delay,date,sched_datetime
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,09:05:00,NaN,NaN,2019-09-16,2019-09-16 09:05:00
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,07:55:00,NaN,NaN,2019-09-16,2019-09-16 07:55:00
2,2019,9,16,월,`,ARP3,L,L1720,A,14:40:00,NaN,NaN,2019-09-16,2019-09-16 14:40:00
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30:00,NaN,NaN,2019-09-16,2019-09-16 13:30:00
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10:00,NaN,NaN,2019-09-16,2019-09-16 20:10:00


# SFSNT.csv

In [99]:
df_sched = pd.read_csv("SFSNT.csv", dtype={"FSD":str, "FED":str}, encoding="CP949")
df_sched = df_sched.drop("SSC", 1)
# print(df_sched.apply(lambda x: np.sum(pd.isnull(x)))) # 결측치 존재 
df_sched.loc[:, ["STA", "STD"]] = df_sched[["STA", "STD"]].fillna("00:00")
df_sched["STA"] = pd.to_datetime(df_sched["STA"], format = "%H:%M").dt.time
df_sched["STD"] = pd.to_datetime(df_sched["STD"], format = "%H:%M").dt.time

df_sched = df_sched.rename(columns={ 
    "ORG":"origin", "DES":"dest", 
    "FLO":"airline", "FLT":"flight",  
    "STD":"dep_time", "STA":"arr_time",  # dep: depart, arr: arrive 
    "FSD":"start_date", "FED":"end_date", 
    "IRR":"is_regular" 
})



df_sched["start_date"] = df_sched['start_date'].str[:4] + "-" + df_sched['start_date'].str[4:6] + "-" + df_sched['start_date'].str[6:]
df_sched["end_date"] = df_sched['end_date'].str[:4] + "-" + df_sched['end_date'].str[4:6] + "-" + df_sched['end_date'].str[6:]
df_sched["start_dep_datetime"] = pd.to_datetime(df_sched["start_date"].astype(str) + " " + df_sched["dep_time"].astype(str))
df_sched["start_arr_datetime"] = pd.to_datetime(df_sched["start_date"].astype(str) + " " + df_sched["arr_time"].astype(str))
df_sched["end_dep_datetime"] = pd.to_datetime(df_sched["end_date"].astype(str) + " " + df_sched["dep_time"].astype(str))
df_sched["end_arr_datetime"] = pd.to_datetime(df_sched["end_date"].astype(str) + " " + df_sched["arr_time"].astype(str))

df_sched.head()

,flight,origin,dest,dep_time,arr_time,airline,MON,TUE,WED,THU,FRI,SAT,SUN,start_date,end_date,is_regular,start_dep_datetime,start_arr_datetime,end_dep_datetime,end_arr_datetime
0,A1915,ARP1,ARP3,07:55:00,09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,2019-03-31 07:55:00,2019-03-31 09:05:00,2019-10-26 07:55:00,2019-10-26 09:05:00
1,A1904,ARP3,ARP1,07:55:00,09:05:00,A,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,2019-03-31 07:55:00,2019-03-31 09:05:00,2019-10-26 07:55:00,2019-10-26 09:05:00
2,I1304,ARP3,ARP1,08:00:00,09:15:00,I,Y,NaN,Y,Y,Y,NaN,Y,2019-03-31,2019-10-26,N,2019-03-31 08:00:00,2019-03-31 09:15:00,2019-10-26 08:00:00,2019-10-26 09:15:00
3,B1802,ARP2,ARP1,08:00:00,08:55:00,B,Y,Y,Y,Y,Y,Y,Y,2019-03-31,2019-10-26,N,2019-03-31 08:00:00,2019-03-31 08:55:00,2019-10-26 08:00:00,2019-10-26 08:55:00
4,J1106,ARP1,ARP2,08:00:00,09:05:00,J,NaN,Y,Y,NaN,Y,Y,NaN,2019-07-30,2019-08-24,N,2019-07-30 08:00:00,2019-07-30 09:05:00,2019-08-24 08:00:00,2019-08-24 09:05:00


일단 요일은 건드리지 않았습니다.